In [52]:
import pandas as pd
import numpy as np
import requests
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper
from bokeh.palettes import Spectral6
import json

output_notebook()

Loading BokehJS ...

In [84]:
str(pd.Timestamp.now()).replace(' ', '+')[0:-7]

'2018-07-20+15:55:55'

In [86]:
payload = {'format': 'geojson', 'limit': 20000, 'orderby': 'time', 'eventtype': 'earthquake', 'starttime': '2014-01-01'}
storage_dict = {'x': [], 'y': [], 'z': [], 'code': [], 'detail': [], 'dmin': [], 'felt': [], 'gap': [],
               'ids': [], 'mag': [], 'magtype': [], 'mmi': [], 'net': [], 'nst': [], 'place': [],
               'rms': [], 'sig': [], 'time': [], 'title': [], 'tsunami': [], 'url': []}
endtime = str(pd.Timestamp.now()).replace(' ', '+')[0:-7]
for i in range(0, 1):
           # https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2014-01-01&endtime=2014-01-02
    url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?endtime={}'.format(endtime)
    r = requests.get(url, params=payload)
    print('{} -> {}'.format(r.status_code, r.url))
    try:
        data_load = json.loads(json.dumps(r.json()))
        dataset = pd.read_json(json.dumps(data_load['features']))
        print(dataset.shape)
        for index, row in dataset.iterrows():
            storage_dict['x'].append(row['geometry']['coordinates'][0]) 
            storage_dict['y'].append(row['geometry']['coordinates'][1])
            storage_dict['z'].append(row['geometry']['coordinates'][2])
            storage_dict['code'].append(row['properties']['code'])
            storage_dict['detail'].append(row['properties']['detail'])
            storage_dict['dmin'].append(row['properties']['dmin'])
            storage_dict['felt'].append(row['properties']['felt'])
            storage_dict['gap'].append(row['properties']['gap'])
            storage_dict['ids'].append(row['properties']['ids'])
            storage_dict['mag'].append(row['properties']['mag'])
            storage_dict['magtype'].append(row['properties']['magType'])
            storage_dict['mmi'].append(row['properties']['mmi'])
            storage_dict['net'].append(row['properties']['net'])
            storage_dict['nst'].append(row['properties']['nst'])
            storage_dict['place'].append(row['properties']['place'])
            storage_dict['rms'].append(row['properties']['rms'])
            storage_dict['sig'].append(row['properties']['sig'])
            storage_dict['time'].append(pd.to_datetime(row['properties']['time'], unit='ms'))
            print(storage_dict['time'][-1])
            storage_dict['title'].append(row['properties']['title'])
            storage_dict['tsunami'].append(row['properties']['tsunami'])
            storage_dict['url'].append(row['properties']['url'])
    #     print(storage_dict['time'][-1])
    #     print(storage_dict['time'][0])
        # payload['endtime'] = storage_dict['time'][-1].strftime('%Y-%m-%dT%H:%M')
        # print('---> {}'.format(payload['endtime']))
    #     time = storage_dict['time'][-1].strftime('%Y-%m-%dT%H:%M:%S')
        endtime = storage_dict['time'][-1]
        print('---- {} DONE ----'.format(i))
    except Exception:
        # payload['endtime'] = storage_dict['time'][-1].strftime('%Y-%m-%dT%H:%M:%S')
        print('oof')
        pass

print(endtime)  

# print('{} <-> {}'.format(data.head(1)['time'], data.tail(1)['time']))

data = pd.DataFrame.from_dict(storage_dict)

data.head()
# r = requests.get('https://earthquake.usgs.gov/fdsnws/event/1/query', params=payload)
# r.url

503 -> https://earthquake.usgs.gov/fdsnws/event/1/query?endtime=2018-07-20+16:18:33&limit=20000&eventtype=earthquake&format=geojson&orderby=time&starttime=2014-01-01
oof
2018-07-20+16:18:33


,code,detail,dmin,felt,gap,ids,mag,magtype,mmi,net,...,place,rms,sig,time,title,tsunami,url,x,y,z


In [58]:
data = data.drop_duplicates(subset=['url'])
data.sort_values(by='time')
data.shape

(26448, 22)

In [28]:
data.to_csv('earthquakes.csv') # save data

In [57]:
data = pd.read_csv('earthquakes.csv')

# for index, row in data.iterrows():
#     print(row['time'])

# data['time'] = data.apply(lambda row: pd.to_datetime(row['time'], unit='ms'), axis=1)
# data['time']

plot = figure(title="Earthquake Intensity over Time (Magnitude)", 
              x_axis_label='Time',
              y_axis_label='Magnitude',
              x_axis_type='datetime',
              y_range=(0,7))

color = ['red', 'blue']

# print(data['tsunami'])

significant = np.interp(data['sig'], (data['sig'].min(), data['sig'].max()), (2, 5))

mapper = LinearColorMapper(palette=Spectral6, low=0, high=100)

mapper.low_color = 'blue'
mapper.high_color = 'red'

# print(data['time'])
# print(data['mag'])

# print('{} --- {}'.format(data['time'].shape, data['mag'].shape))

plot.scatter(pd.to_datetime(data['time']), data['mag'], fill_alpha=0.4, line_alpha=0, fill_color={'field': 'y', 'transform': mapper}, size=significant)

# plot.scatter(data['time'], data['mag'], fill_alpha=0.4, line_alpha=1, color='blue', size=significant)
# plot.scatter(dataset['Date'], dataset['MMI'], size=injuries, fill_alpha=0.2, line_alpha=0, color='red')

show(plot)